In [2]:
import pandas as pd           # data mnipulation
import numpy as np            # number manipulation/crunching
import matplotlib.pyplot as plt  # plotting
from sklearn.metrics import  accuracy_score, classification_report 
import openpyxl
import sklearn
# Train Test split
from sklearn.model_selection import train_test_split
# Random forest classifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from anchor import utils
from anchor import anchor_tabular
from sklearn.model_selection import LeaveOneOut
import lime
import lime.lime_tabular
from collections import Counter
import shap
import math

hepatitis = pd.read_csv("hepatitis.csv")
hepatitis['ALB'] = hepatitis['ALB'].replace(to_replace=np.nan,value = hepatitis['ALB'].median())
hepatitis['ALP'] = hepatitis['ALP'].replace(to_replace=np.nan,value = hepatitis['ALP'].median())
hepatitis['ALT'] = hepatitis['ALT'].replace(to_replace=np.nan,value = hepatitis['ALT'].median())
hepatitis['AST'] = hepatitis['AST'].replace(to_replace=np.nan,value = hepatitis['AST'].median())
hepatitis['BIL'] = hepatitis['BIL'].replace(to_replace=np.nan,value = hepatitis['BIL'].median())
hepatitis['CHE'] = hepatitis['CHE'].replace(to_replace=np.nan,value = hepatitis['CHE'].median())
hepatitis['CHOL'] = hepatitis['CHOL'].replace(to_replace=np.nan,value = hepatitis['CHOL'].median())
hepatitis['CREA'] = hepatitis['CREA'].replace(to_replace=np.nan,value = hepatitis['CREA'].median())
hepatitis['GGT'] = hepatitis['GGT'].replace(to_replace=np.nan,value = hepatitis['GGT'].median())
hepatitis['PROT'] = hepatitis['PROT'].replace(to_replace=np.nan,value = hepatitis['PROT'].median())

hepatitis['Outcome'].value_counts()
Y = hepatitis['Outcome']
target_names=["0", "1"]
    
X = hepatitis[['Age',	'Sex',	'ALB',	'ALP',	'ALT',	'AST',	'BIL',	'CHE',	'CHOL',	'CREA',	'GGT',	'PROT']]
maximums = np.amax(np.array(X), axis=0)
#X = diabetes.iloc[:,:-1].values
X_featurenames = X.columns

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)
counter = Counter(Y)
model = sklearn.svm.SVC(kernel='rbf', probability=True)
model.fit(X_train, Y_train)
pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, pred)
print("Accuracy:", accuracy)
print(classification_report(Y_test, pred, target_names=["0", "1"]))

# model = KNeighborsClassifier(n_neighbors=11)
# model.fit(X_train, Y_train)
# predicted_knn = model.predict(X_test)
# accuracy_knn = accuracy_score(Y_test, predicted_knn)
# print("Precisión Knn ", accuracy_knn)

shap.initjs()

dataframe_columns_Shap = []
test_index = "test_index"
dataframe_columns_Shap.append(test_index)
for feature in X_featurenames:
    feature_shap_value = feature + "_shap_value"
    feature_sign = feature + "_sign"
    feature_rank = feature + "_rank"
    dataframe_columns_Shap.append(feature_shap_value)
    dataframe_columns_Shap.append(feature_sign)
    dataframe_columns_Shap.append(feature_rank)
expected_value_0 = "expected_value_0"
expected_value_1 = "expected_value_1"
f_0 = "f_0"
f_1 = "f_1"
probability_no_hepatitis = "probability_no_hepatitis"
probability_hepatitis  = "probability_hepatitis "
accuracy_classifier = "accuracy_classifier"
true_class = "true_class"
dataframe_columns_Shap.append(expected_value_0)
dataframe_columns_Shap.append(expected_value_1)
dataframe_columns_Shap.append(f_0)
dataframe_columns_Shap.append(f_1)
dataframe_columns_Shap.append(probability_no_hepatitis )
dataframe_columns_Shap.append(probability_hepatitis )
dataframe_columns_Shap.append(accuracy_classifier)
dataframe_columns_Shap.append(true_class)
df_Shap = pd.DataFrame(columns = dataframe_columns_Shap)


cv = LeaveOneOut()
number_iterations = 1

for train_ix, test_ix in cv.split(X):
    print(test_ix[0])
    for count in range(number_iterations):
        row_Shap = []
        row_Shap.append(test_ix[0])
        model = sklearn.svm.SVC(kernel='rbf', probability=True)
        X_train, X_test = X.to_numpy()[train_ix, :], X.to_numpy()[test_ix, :]
        Y_train, Y_test = Y[train_ix], Y[test_ix]
        model.fit(X_train, Y_train)
        # a summarized version of the dataset using shap.kmeans. This has been done to reduce computation time
        # explainer = shap.KernelExplainer(model.predict_proba, shap.kmeans(X_train, 10))
        explainer = shap.KernelExplainer(model.predict_proba, X_train)
        shap_values = explainer.shap_values(X_test)
        #print(shap_values)
        shap_values = np.delete(shap_values, obj = 1, axis = 0).flatten()
        #print(shap_values)
        predict_proba = model.predict_proba(X_test).flatten()
        #print(predict_proba)
        ranks = []
        for i in range(len(X_featurenames)):
            count_lower = 0
            for j in range(len(X_featurenames)):
                if abs(shap_values[j]) < abs(shap_values[i]):
                    count_lower = count_lower + 1
            position = len(X_featurenames) - count_lower
            row_Shap.append(shap_values[i])
            if shap_values[i] > 0 :
                sign = "positive"
            else :
                sign = "negative"
            row_Shap.append(sign)
            row_Shap.append(position)
            ranks.append(position)
        #print(ranks)
        #print(X_featurenames)
        #print(explainer.expected_value)
        row_Shap.append(explainer.expected_value[0])
        row_Shap.append(explainer.expected_value[1])
        sum_shap_values = np.array(shap_values).sum(0) 
        f_0 = explainer.expected_value[0] + sum_shap_values
        f_1 = explainer.expected_value[1] - sum_shap_values
        #print(f_0)
        #print(f_1)
        row_Shap.append(f_0)
        row_Shap.append(f_1)
        probability_no_hepatitis  = predict_proba[0]
        probability_hepatitis  = predict_proba[1]
        row_Shap.append(probability_no_hepatitis )
        row_Shap.append(probability_hepatitis )
        pred = model.predict(X_test)
        accuracy_classifier = accuracy_score(Y_test, pred)
        true_class = target_names[Y_test[test_ix[0]]]
        row_Shap.append(accuracy_classifier)
        row_Shap.append(true_class)
        print(row_Shap)
        df_Shap.loc[test_ix[0] * number_iterations + count] = row_Shap
        
df_Shap.to_excel("outputHepatitisKernelShapLOOCV.xlsx")

Accuracy: 0.943089430894309
              precision    recall  f1-score   support

           0       0.94      0.99      0.97       103
           1       0.93      0.70      0.80        20

    accuracy                           0.94       123
   macro avg       0.94      0.85      0.88       123
weighted avg       0.94      0.94      0.94       123



Using 614 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


0


  0%|          | 0/1 [00:00<?, ?it/s]

Using 614 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


...

  0%|          | 0/1 [00:00<?, ?it/s]

Using 614 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


[613, 0.002506430165712512, 'positive', 8, 0.00023086668950402867, 'positive', 10, -0.0020909070763466897, 'negative', 9, 0.019692102283566697, 'positive', 6, 0.08041820007956851, 'positive', 2, -0.27027286211808965, 'negative', 1, -0.03476154039136647, 'negative', 5, -0.00017066336537147956, 'negative', 11, -4.333002582561269e-05, 'negative', 12, -0.05119870362701671, 'negative', 4, -0.07071301746428615, 'negative', 3, 0.004675765829666667, 'positive', 7, 0.879589995658549, 0.12041000434145091, 0.5578623366382647, 0.4421376633617352, 0.5578623366382647, 0.44213766336173527, 0.0, '1']
614


  0%|          | 0/1 [00:00<?, ?it/s]

[614, 0.0029157132568745094, 'positive', 9, -0.00016027202520877554, 'negative', 11, -0.0038199958903394594, 'negative', 8, 0.004518303399718827, 'positive', 7, 0.03389014188950444, 'positive', 2, -0.5217687703927343, 'negative', 1, -0.00492139883898407, 'negative', 6, 0.00013482365370606444, 'positive', 12, 0.0007561786243052691, 'positive', 10, -0.02079907320770947, 'negative', 3, -0.020453190705861237, 'negative', 4, 0.016606712922650035, 'positive', 5, 0.8804597403452881, 0.11954025965471211, 0.3673589130312098, 0.6326410869687904, 0.36735891303120977, 0.6326410869687902, 0.0, '1']
